In [3]:
import pandas as pd

# Пример DataFrame
df = pd.read_csv('/home/jupyter/datasphere/project/ja_ready_data.csv')

# Найдем строки с дубликатами по столбцу 'text'
df_unique = df.drop_duplicates(subset=['text'], keep='first')

print("Оригинальный DataFrame:")
print(len(df))

print("DataFrame без строк, у которых есть дубликаты по столбцу 'text':")
print(len(df_unique))

Оригинальный DataFrame:
108732
DataFrame без строк, у которых есть дубликаты по столбцу 'text':
108430


In [5]:
def char_count(text):
    if isinstance(text, str):
        return len(text)
    else:
        return 0 
ddd = df_unique.copy()

ddd.loc[:, 'char_count'] = ddd['text'].apply(char_count)

# Находим тексты короче 20 символов
short_texts = ddd[ddd['char_count'] < 20]

# Сохраняем их индексы
short_texts_indices = short_texts.index.tolist()

# Удаляем
ddd = ddd.drop(index=short_texts_indices)

print("До удаления коротких:", len(df_unique))
print("После:", len(ddd))

До удаления коротких: 108430
После: 108210


In [6]:
import unicodedata
# Функции определения символов и подсчета их соотношения 
def is_latin(character):
    if not character.isalpha():
        return False
    try:
        block_name = unicodedata.name(character)
        return 'LATIN' in block_name
    except ValueError:
        return False

def is_cjk(character):
    try:
        block_name = unicodedata.name(character)
        return any(keyword in block_name for keyword in ['CJK UNIFIED IDEOGRAPH', 'CJK COMPATIBILITY IDEOGRAPH', 
                                                         'CJK RADICAL', 'HIRAGANA', 'KATAKANA', 'KANJI'])
    except ValueError:
        return False

def determine_script_type(text):
    if not isinstance(text, str):
        return "ooops"
    
    latin_count = 0
    cjk_count = 0
    non_latin_non_numeric_count = 0
    
    for char in text:
        if is_latin(char):
            latin_count += 1
        elif is_cjk(char):
            cjk_count += 1
        elif not char.isdigit():
            non_latin_non_numeric_count += 1
  
    
    total_chars = len(text)
 
    if cjk_count < 0.40 * total_chars:    # если иероглифов меньше 40% - удаляем
        return "ooops"
    else:
        return "ok"

# Пример использования
print(determine_script_type("徳島県で9月14日～21日の8日間で3件3人の交通死亡事故が発生したことを受け、徳島県交通安全対策協議会は22日、今年6回目となる「交通死亡事故多発警報(全県警報)」を発令し、10月1日まで街頭活動や広報活動等の緊急対策を推進しています。なお、現在の発令基準となった2016年以降では年間で最多となっています。 なお23日現在、徳島県内での2019年交通事故死者は計31人(前年同期比＋8)で、264日目で2018年の年間総数と同数に達しました。都道府県別での交通事故死者前年総数到達は滋賀県(8/12)、鳥取県(8/26)に次いで3県目です。また、うち65歳以上の高齢者は22人(＋12)と7割超を占めています。 【徳島県内での2019年交通事故死者数】(9月23日現在：速報値) ・ 1月：3人(＋3)　うち高齢者2人(＋2) ・ 2月：5人(＋3)　　　　　　4人(＋4)　多発警報 02/07～/16 ・ 3月：5人(＋1)　　　　　　3人(＋1)　多発警報 03/20～/29 ・ 4月：4人(＋3)　　　　　　3人(＋2)　多発警報 04/09～/18 ・ 5月：4人(＋2)　　　　　　3人(＋2)　多発警報 05/12～/21 ・ 6月：1人(－3)　　　　　　1人(±0) ・ 7月：5人(＋3)　　　　　　2人(±0) ・ 8月：1人(－4)　　　　　　1人(－2)　多発警報 08/02～/11 ・ 9月：3人(±0)　　　　　　3人(＋3) ・8.58日に1人の割合(上期8.22日/下期9.44日) ・都道府県別：総数29位 増減数5位 増減率5位 人口10万人比1位 ・年間死者数：2018年31人 17年34人 16年49人 15年27人 14年31人 　うち高齢者：　　　18人 　　19人 　　38人 　　17人 　　19人"))

ok


In [8]:
# Применяем функцию к столбцу 'text' и сохраняем результат в новый столбец 'if_ooops'
ddd['if_ooops'] = ddd['text'].apply(determine_script_type)

short_texts = ddd[ddd['if_ooops'] == 'ooops']

indexes_to_remove = short_texts.index.tolist()

# Смотрим, сколько текстов будет удалено
len(indexes_to_remove)

18445

In [9]:
print("Оригинальный DataFrame:")
print(len(ddd))

without = ddd.drop(indexes_to_remove).reset_index(drop=True)

print("Очищенный")
print(len(without))

Оригинальный DataFrame:
108210
Очищенный
89765


In [59]:
without.to_csv('ja_before.csv')